In [ ]:
# 检测 chat_rag.py 文件
file_path = "../app_image.py"
# file_path = r"C:\Users\A\Project_InputCheck\test.py"
!flake8 {file_path} --max-line-length=240
!pylint {file_path}

In [ ]:
%%writefile ..\app.py
# pylint: disable=no-member  # Project structure requires dynamic path handling
"""
For more information on `huggingface_hub` Inference API support
please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
import os
import sys
import gradio as gr
import cv2
from pyzbar.pyzbar import decode
import numpy as np
from google import genai
from google.genai import types
from dotenv import load_dotenv

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

with open(
    os.path.join(current_dir, "system_role_prompt.md"), "r", encoding="utf-8"
) as f:
    system_role = f.read()


BEGIN_PROMPT = """
总结一下最新的内容
"""


CONFIRM_PROMPT = """
对比一下两个版本的差异
"""

load_dotenv(dotenv_path=os.path.join(current_dir, ".env"))  # current_dir + "\.env")
api_key = os.getenv("GEMINI_API_KEY")
gemini_client = None
if api_key:
    gemini_client = genai.Client(api_key=api_key)

MODEL_NAME = "gemini-2.0-flash-exp"


# Common safety settings for all requests
def get_safety_settings():
    return [
        types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_CIVIC_INTEGRITY", threshold="OFF")
    ]


def format_content(role: str, text: str) -> types.Content:
    """格式化单条消息内容"""
    return types.Content(
        role=role,
        parts=[types.Part(text=text)]
    )


def respond(
    message,
    history: list[tuple[str, str]],
    use_system_message,
):
    # 构建对话历史
    def build_contents(message=None, before_message=None):
        contents = []
        for val in history:
            if val["content"] == "开始":
                context = BEGIN_PROMPT
            elif val["content"] == "确认":
                context = CONFIRM_PROMPT
            else:
                context = val["content"]
            contents.append(format_content(
                val["role"],
                context
            ))

        if before_message:
            contents.append(format_content(
                "assistant",
                before_message
            ))

        if message:
            contents.append(format_content(
                "user",
                message
            ))
        return contents
    if message == "开始" and not history:
        message = BEGIN_PROMPT

    if message == "确认" and len(history) == 2:
        message = CONFIRM_PROMPT

    if message:
        # 处理普通消息
        contents = build_contents(message)
        response = ""
        if use_system_message:
            config = types.GenerateContentConfig(
                system_instruction=system_role,
                safety_settings=get_safety_settings(),
            )
        else:
            config = types.GenerateContentConfig(
                safety_settings=get_safety_settings(),
            )
        for chunk in gemini_client.models.generate_content_stream(
            model=MODEL_NAME,
            contents=contents,
            config=config
        ):
            if chunk.text:  # Check if chunk.text is not None
                response += chunk.text
                yield response


def get_gradio_version():
    return gr.__version__


game_state = {"gr_version": get_gradio_version()}  # 示例 game_state


def process_qr_frame(frame):
    """处理视频帧，检测和解码二维码"""
    if frame is None:
        return frame, game_state

    # 转换图像格式确保兼容性
    if isinstance(frame, np.ndarray):
        img = frame.copy()
    else:
        img = np.array(frame).copy()

    # 检测二维码
    decoded_objects = decode(img)

    if decoded_objects:
        # 获取二维码数据
        qr_data = decoded_objects[0].data.decode('utf-8')

        # 解析二维码数据
        qr_info = {"设定": qr_data}

        game_state.update(qr_info)

        # 在图像上绘制识别框和状态
        points = decoded_objects[0].polygon
        if points:
            # 绘制绿色边框表示成功识别
            pts = np.array(points, np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(img, [pts], True, (0, 255, 0), 2)

            # 添加文本显示已更新
            cv2.putText(img, "QR Code Updated!", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return img, game_state


def export_chat(history):
    export_string = ""
    for item in history:
        user_msg = item[0]['content']
        bot_msg = item[1]
        export_string += f"User: {user_msg}\nBot: {bot_msg}\n\n"
    return export_string


def get_chat_history(chatbot_component):
    return chatbot_component.load_history()


with gr.Blocks(theme="soft") as demo:
    if gemini_client:
        chatbot = gr.ChatInterface(
            respond,
            title="知识助理",
            type="messages",
            additional_inputs=[
                gr.Checkbox(value=False, label="Use system message"),
            ],

        )
    else:
        gr.Markdown("Gemini API key not found. Please check your .env file.")

    with gr.Accordion("查看状态", open=False):
        game_info_image = gr.Image(label="二维码设定",
                                   webcam_constraints={
                                        "video": {
                                            "facingMode": {"ideal": "environment"}
                                        }
                                    })
        output_img = gr.Image(label="识别结果")
        game_state_output = gr.JSON(value=game_state)  # 初始显示 game_state

        # 设置实时流处理
        game_info_image.upload(
            fn=process_qr_frame,
            inputs=[game_info_image],
            outputs=[output_img, game_state_output],
            show_progress=False,
        )
        # 设置实时流处理
        game_info_image.stream(
            fn=process_qr_frame,
            inputs=[game_info_image],
            outputs=[output_img, game_state_output],
            show_progress=False,
            stream_every=0.5  # 每0.5秒处理一次
        )

if __name__ == "__main__":
    cert_file = os.path.join(current_dir, "localhost+1.pem")
    key_file = os.path.join(current_dir, "localhost+1-key.pem")

    if os.path.exists(cert_file) and os.path.exists(key_file):
        demo.launch(
            server_name="0.0.0.0",
            ssl_certfile=cert_file,
            ssl_keyfile=key_file
        )
    else:
        demo.launch(server_name="0.0.0.0")


In [ ]:
%%writefile ..\app_image.py
# pylint: disable=no-member  # Project structure requires dynamic path handling
"""
whisk逆向图片生成
"""
import os
import sys
import hashlib
import json
from datetime import datetime
from typing import Optional, List, Dict
from requests.exceptions import HTTPError
from dotenv import load_dotenv
import gradio as gr

# ===== 2. 初始化配置 =====
# 获取当前文件所在目录的绝对路径
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.normpath(os.path.join(current_dir, ".."))
else:
    # 在 Jupyter Notebook 环境中
    current_dir = os.getcwd()
    current_dir = os.path.join(current_dir, "..")
    root_dir = os.path.normpath(os.path.join(current_dir))

current_dir = os.path.normpath(current_dir)
sys.path.append(current_dir)

from Module.Common.scripts.llm.utils.google_whisk import (
    generate_image_base64,
    generate_caption,
    generate_image_fx,
    generate_story_board,
    DEFAULT_STYLE_PROMPT_DICT,
    DEFAULT_HEADERS,
    AspectRatio,
    Category
)
from Module.Common.scripts.common.auth_manager import (
    AuthKeeper,
    sustain_auth
)
from Module.Common.scripts.datasource_feishu import FeishuClient

CONFIG_PATH = os.path.join(current_dir, "config.json")

# 加载配置文件
with open(CONFIG_PATH, 'r', encoding='utf-8') as config_file:
    CONFIG = json.load(config_file)

# 修改后的常量定义
IMAGE_NUMBER = CONFIG["image_generation"]["default_image_number"]
ASPECT_RATIO = AspectRatio[CONFIG["image_generation"]["aspect_ratio"]]
SERVER_PORT = CONFIG["server"]["port"]
ERROR_PATTERNS = CONFIG["error_patterns"]
FEISHU_RECEIVE_ID = CONFIG["feishu"]["receive_id"]

# 缓存文件路径
CACHE_DIR = os.path.join(current_dir, CONFIG["cache"]["dir"])
CAPTION_CACHE_FILE = os.path.join(CACHE_DIR, CONFIG["cache"]["caption_file"])
STORY_CACHE_FILE = os.path.join(CACHE_DIR, CONFIG["cache"]["story_file"])
IMAGE_CACHE_DIR = os.path.join(CACHE_DIR, CONFIG["cache"]["image_dir"])


load_dotenv(os.path.join(current_dir, ".env"))
app_id = os.getenv("FEISHU_APP_ID", "")
app_secret = os.getenv("FEISHU_APP_SECRET", "")

# 改进后的飞书客户端初始化（更Pythonic的写法）
FEISHU_CLIENT = None
FEISHU_ENABLED = False
if app_id and app_secret:  # 先做基础检查
    try:
        FEISHU_CLIENT = FeishuClient(app_id, app_secret)
        FEISHU_CLIENT.get_access_token()  # 主动验证凭证有效性
        FEISHU_ENABLED = True
    except (ValueError, RuntimeError) as e:
        print(f"⚠️ 飞书通知功能已禁用: {str(e)}")
else:
    print("⚠️ 飞书通知功能已禁用: 缺少FEISHU_APP_ID或FEISHU_APP_SECRET环境变量")


class WhiskCache:
    """Whisk缓存管理类"""
    def __init__(self):
        os.makedirs(IMAGE_CACHE_DIR, exist_ok=True)
        self.caption_cache = self._load_cache(CAPTION_CACHE_FILE)
        self.story_cache = self._load_cache(STORY_CACHE_FILE)

    def _load_cache(self, file_path: str) -> Dict:
        """加载缓存文件"""
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}

    def save_cache(self, file_path: str, data: Dict):
        """保存缓存到文件"""
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def get_caption(self, image_base64: str):
        """获取缓存的图片描述"""
        hash_key = hashlib.md5(image_base64.encode()).hexdigest()
        return self.caption_cache.get(hash_key)

    def save_caption(self, image_base64: str, caption: str):
        """缓存图片描述"""
        hash_key = hashlib.md5(image_base64.encode()).hexdigest()
        self.caption_cache[hash_key] = caption
        self.save_cache(CAPTION_CACHE_FILE, self.caption_cache)

    def get_story_prompt(self, caption: str, style_key: str, additional_text: str):
        """获取缓存的故事提示词"""
        cache_key = hashlib.md5(f"{caption}_{style_key}_{additional_text}".encode()).hexdigest()
        return self.story_cache.get(cache_key)

    def save_story_prompt(self, caption: str, style_key: str, additional_text: str, prompt: str):
        """缓存故事提示词"""
        cache_key = hashlib.md5(f"{caption}_{style_key}_{additional_text}".encode()).hexdigest()
        self.story_cache[cache_key] = prompt
        self.save_cache(STORY_CACHE_FILE, self.story_cache)


class WhiskService:
    """Whisk服务类"""
    def __init__(self):
        self.cache = WhiskCache()
        self.auth_keeper = AuthKeeper(
            config_path=os.path.join(current_dir, "auth_config.json"),
            default_headers=DEFAULT_HEADERS
        )
        # 保持原始装饰器调用方式
        self.sustain_cookies = sustain_auth(self.auth_keeper, 'cookies')
        self.sustain_token = sustain_auth(self.auth_keeper, 'auth_token')

    @property
    def generate_caption_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_cookies(self._generate_caption_impl)

    def _generate_caption_impl(
        self,
        image_base64: str,
        category: Category = Category.CHARACTER,  # 补全参数
        cookies: str = None  # 保持装饰器参数
    ) -> Dict:
        return generate_caption(
            image_base64=image_base64,
            category=category,  # 传递补全参数
            cookies=cookies
        )

    @property
    def generate_story_board_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_cookies(self._generate_story_board_impl)

    def _generate_story_board_impl(
        self,
        characters: Optional[List[str]] = None,
        style_prompt: Optional[str] = None,
        location_prompt: Optional[str] = None,  # 补全参数
        pose_prompt: Optional[str] = None,       # 补全参数
        additional_input: str = "",
        cookies: str = None
    ) -> Dict:
        return generate_story_board(
            characters=characters,
            style_prompt=style_prompt,
            location_prompt=location_prompt,  # 传递补全参数
            pose_prompt=pose_prompt,           # 传递补全参数
            additional_input=additional_input,
            cookies=cookies
        )

    @property
    def generate_image_fx_wrapped(self):
        """保持装饰器调用方式不变"""
        return self.sustain_token(self._generate_image_fx_impl)

    def _generate_image_fx_impl(
        self,
        prompt: str,
        seed: Optional[int] = None,
        aspect_ratio: AspectRatio = AspectRatio.LANDSCAPE,
        output_prefix: str = None,
        image_number: int = 4,
        auth_token: str = None,
        save_local: bool = False  # 补全参数
    ) -> List:
        return generate_image_fx(
            prompt=prompt,
            seed=seed,
            aspect_ratio=aspect_ratio,
            output_prefix=output_prefix,
            image_number=image_number,
            auth_token=auth_token,
            save_local=save_local  # 传递补全参数
        )

    def generate_images(
        self,
        image_input1: str,
        image_input2: str,
        style_key: str,
        additional_text: str
    ):
        """处理图片生成请求"""
        # 基础参数准备
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        current_date = datetime.now().strftime("%Y%m%d")
        output_prefix = os.path.join(
            IMAGE_CACHE_DIR,
            f"generated_image_{current_date}_take{len(os.listdir(IMAGE_CACHE_DIR)) + 1}"
        )

        # 处理指令直出模式
        if additional_text.startswith('/img') and len(additional_text) > 4:
            clean_prompt = additional_text[4:].strip()
            if clean_prompt:
                return self._handle_direct_mode(clean_prompt, current_time, output_prefix)

        # 输入检查
        if not any([image_input1, image_input2]):
            return None, None

        # 预处理图片数据
        image_data = self._prepare_image_data(image_input1, image_input2)
        all_caption_cached = all(caption for _, caption in image_data)

        # 打印缓存状态
        self._print_cache_status(current_time, all_caption_cached, image_data, style_key, additional_text)

        try:
            # 生成描述和故事板
            captions = self._process_captions(image_data)
            if not captions:
                return None, None

            # 生成最终提示词
            final_prompt = self._get_final_prompt(captions, style_key, additional_text)
            print(f"最终Prompt: \n{final_prompt}")

            return self._generate_final_images(final_prompt, output_prefix)

        except HTTPError as e:
            return self._handle_http_error(e)
        except Exception as e:
            print(f"其他图片生成错误: {str(e)}")
            return None, None

    def _handle_direct_mode(self, clean_prompt, current_time, output_prefix):
        """处理指令直出模式"""
        print(f"[{current_time}] 指令直出，使用提示词: \n{clean_prompt}")
        try:
            image_files = self.generate_image_fx_wrapped(
                prompt=clean_prompt,
                output_prefix=output_prefix,
                image_number=2
            )
            return self._format_image_output(image_files)
        except HTTPError as e:
            return self._handle_http_error(e)
        except Exception as e:
            print(f"其他图片生成错误: {str(e)}")
            return None, None

    def _prepare_image_data(self, *image_inputs):
        """预处理图片数据"""
        image_data = []
        for img in image_inputs:
            if img:
                base64_str = generate_image_base64(img)
                cached_caption = self.cache.get_caption(base64_str)
                image_data.append((base64_str, cached_caption))
        return image_data

    def _print_cache_status(self, current_time, all_caption_cached, image_data, style_key, additional_text):
        """打印缓存状态"""
        story_prompt_cached = False
        if all_caption_cached:
            caption_text = "|".join(caption for _, caption in image_data)
            story_prompt_cached = self.cache.get_story_prompt(caption_text, style_key, additional_text) is not None

        print(
            f"\n[{current_time}] 生成图片 | "
            f"素材提示词缓存: {'启用' if all_caption_cached else '未启用'} | "
            f"最终提示词缓存: {'启用' if story_prompt_cached else '未启用'}"
        )

    def _process_captions(self, image_data):
        """处理图片描述"""
        captions = []
        for base64_str, cached_caption in image_data:
            if not cached_caption:
                new_caption = self.generate_caption_wrapped(base64_str)
                if new_caption:
                    self.cache.save_caption(base64_str, new_caption)
                    captions.append(new_caption)
            else:
                captions.append(cached_caption)
        return captions

    def _get_final_prompt(self, captions, style_key, additional_text):
        """获取最终提示词"""
        final_prompt = self.cache.get_story_prompt(
            "|".join(captions),
            style_key,
            additional_text
        )
        if not final_prompt:
            final_prompt = self.generate_story_board_wrapped(
                characters=captions,
                style_prompt=DEFAULT_STYLE_PROMPT_DICT.get(style_key, ""),
                additional_input=additional_text
            )
            if final_prompt:
                self.cache.save_story_prompt(
                    "|".join(captions),
                    style_key,
                    additional_text,
                    final_prompt
                )
        return final_prompt

    def _generate_final_images(self, final_prompt, output_prefix):
        """生成最终图片"""
        image_files = self.generate_image_fx_wrapped(
            prompt=final_prompt,
            output_prefix=output_prefix,
            image_number=IMAGE_NUMBER
        )
        return self._format_image_output(image_files)

    def _format_image_output(self, image_files):
        """格式化图片输出"""
        return (
            image_files[0] if image_files else None,
            image_files[1] if len(image_files) > 1 else None
        )

    def _handle_http_error(self, e):
        """统一处理HTTP错误"""
        error_info = {
            "status_code": e.response.status_code,
            "reason": e.response.reason,
            "url": e.response.url,
            "response_text": e.response.text[:200]
        }
        error_source = self._detect_error_source(e.response.url)

        if error_info['status_code'] == 401:
            print(f"[{error_source}] 自动续签认证失败，详细见飞书")
            if FEISHU_ENABLED:
                FEISHU_CLIENT.send_message(
                    receive_id=FEISHU_RECEIVE_ID,
                    content={"text": "Whisk的Cookie已过期，请及时续签"},
                    msg_type="text"
                )
        elif error_info['status_code'] == 400:
            print(f"[{error_source}] 业务和谐，计划重试...")
            gr.Warning("图片生成失败，请换一张再试试")
        else:
            print(f"[{error_source}] 其他HTTP错误，添加一个飞书消息...")

        return None, None

    def _detect_error_source(self, url):
        """优化后的错误来源检测（直观简洁版）"""
        return next(
            (v for k, v in ERROR_PATTERNS.items() if k in url),
            '未知来源'
        )


# 创建服务实例
whisk_service = WhiskService()

# Gradio界面
with gr.Blocks(theme="soft") as demo:
    with gr.Row():
        with gr.Column(scale=1):
            with gr.Row():
                input_image1 = gr.Image(
                    label="上传图片1",
                    type="filepath",
                    height=300
                )
                input_image2 = gr.Image(
                    label="上传图片2（可选）",
                    type="filepath",
                    height=300
                )
            style_dropdown = gr.Dropdown(
                choices=list(DEFAULT_STYLE_PROMPT_DICT.keys()),
                value=list(DEFAULT_STYLE_PROMPT_DICT.keys())[0],
                label="选择风格"
            )
            additional_text_ui = gr.Textbox(
                label="补充提示词",
                placeholder="请输入额外的提示词...",
                lines=3
            )
            generate_btn = gr.Button("生成图片")

        with gr.Column(scale=2):
            output_image1 = gr.Image(label="生成结果 1")
            output_image2 = gr.Image(label="生成结果 2")

    generate_btn.click(
        fn=whisk_service.generate_images,
        inputs=[input_image1, input_image2, style_dropdown, additional_text_ui],
        outputs=[output_image1, output_image2]
    )

if __name__ == "__main__":
    demo.launch(
        server_name=CONFIG["server"]["name"],
        server_port=SERVER_PORT,      # 使用配置值
        ssl_verify=CONFIG["server"]["ssl_verify"],
        share=CONFIG["server"]["share"],
        allowed_paths=[IMAGE_CACHE_DIR]
    )


In [ ]:
valid_json_str = '{"prompt": "A whimsical close-up shot of a bento box. Inside, a miniature young woman, crafted from colored rice and vegetables, with a single sesame seed for an eye, is shown saluting.  Her kimono, rendered in shades of pastel red with a white floral pattern, has long, flowing sleeves.  A tiny pink heart-shaped sign with miniature Japanese characters is in front of her.  The woman\'s dark hair is neatly pulled back, adorned with a small pink flower. Her expression is pleasant. The bento box sits on a table, the surrounding environment outside the box is not visible. The overall style is kawaii, with soft pastel colors and delicate details. The lighting is soft and diffused.  All elements are miniature and made of edible foods, kept entirely within the bento box"}'
import json
# 可以成功解析
parsed = json.loads(valid_json_str)
json.dumps(parsed)
# print(parsed["prompt"][:30])  # 输出前30个字符验证

In [ ]:
# 在这里，您可以通过 ‘args’  获取节点中的输入变量，并通过 'ret' 输出结果
# 'args' 和 'ret' 已经被正确地注入到环境中
# 下面是一个示例，首先获取节点的全部输入参数params，其次获取其中参数名为‘input’的值：
# params = args.params;
# input = params.input;
# 下面是一个示例，输出一个包含多种数据类型的 'ret' 对象：
# ret: Output =  { "name": ‘小明’, "hobbies": [“看书”, “旅游”] };
import requests_async as requests

# 公共配置
DEFAULT_HEADERS = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'zh-CN,zh;q=0.9,ja;q=0.8,zh-TW;q=0.7,en;q=0.6',
    'cache-control': 'no-cache',
    'content-type': 'text/plain;charset=UTF-8',
    'origin': 'https://labs.google',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://labs.google/',
    'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'x-client-data': 'CKu1yQEIlLbJAQiitskBCKmdygEIuoXLAQiVocsBCIqjywEIhaDNAQjU284BCJDfzgEYj87NAQ=='
}

DEFAULT_STYLE_PROMPT_DICT = {
    "贺卡": """
A rectangular shaped Valentine's Day card depicting a layered papercut diorama of a dreamy, romantic scene at sunset with a cute rim embellished with ribbons.
Scene and subject should be made out of paper, ribbons, and stickers. No other materials allowed. Incorporate ALL SUBJECTS and ALL DETAILS FROM THE SCENE.

The color palette is soft and romantic, with pastel pinks, purples, and oranges dominating the sky.
The figures are small and cute, and the overall style is whimsical and charming.
The card is heavily embellished with glitter, adding to the romantic and festive atmosphere.

The final output should look like a card that was handcrafted with love and include a cute Valentine's day message.
""",
    "巧克力": """
A Valentine's day box of chocolates is shown in a close-up shot.
The chocolates are arranged in a dark brown plastic tray with individual compartments.
The chocolates are various shapes and sizes, and are decorated with different colors and patterns.
The lighting is soft and diffused, and the overall tone of the image is warm and inviting.
The image has a slightly desaturated look, with muted colors and a slightly grainy texture.

Exquisitely sculpt all characters as a cute, small, round, chocolate nestled into the box.
If provided a scene or environment, the chocolates are painted and made into creative shapes to depict elements of the scene in detail SEPARATE from the characters.
The characters are positioned within the chocolate box, surrounded by other beautiful chocolates, creating a visually appealing and balanced composition.
The final product should show ONLY a close up of the exquisite and colorful box of decorated chocolates.
""",
    "便当": """
A close-up shot of a super cute bento box scene featuring miniature, real, colorful Japanese foods and veggies.
Characters have single sesame seeds for eyes and are crafted out of different colored rice and veggies to be tiny, kawaii pieces nestled INTO the box.
The characters are positioned centrally within the bento box, surrounded by other miniature food items, creating a visually appealing and balanced composition.

Elements must be mini and created by edible foods only and always kept INSIDE the bento box.
If provided, incorporate a scene/environment OUTSIDE of the box. Make it such that the box sits on a table in that location.
The overall style is whimsical and charming, with a focus on soft, pastel colors and delicate details. The lighting is soft and diffused, enhancing the delicate textures and colors.

The final product should show ONLY a close up of the exquisite bento box sitting on a surface of a given location.
""",
}


def generate_imagen3_character_prompt(
    characters_descriptions=None,
    location=None,
    style_description=None,
    pose_description=None,
    user_instructions=None
) -> str:
    prompt_parts = []

    # 添加基础开头
    base_start = """你是一个提示词大师，请根据下面的信息制作一个提供给图片生成模型的提示词。
Instructions:
1. You will be creating a prompt for a text-to-image model that will be placing characters/subjects into a location background.
2. You will make sure to concisely describe each character/subject and what they are doing and will not lose track of the characters'/subjects' visual details.
3. You will make sure the background is the one provided by the location prompt."""
    prompt_parts.append(base_start)

    # 处理角色描述
    if characters_descriptions and len(characters_descriptions) > 0:
        character_template = """
There are {num_characters} characters/subjects in the scene. Make sure to ignore the locations from these character/subject prompts and only extract the character/subject:
{character_prompts}"""

        character_prompts = []
        for i, desc in enumerate(characters_descriptions, 1):
            character_prompts.append(f"Character/Subject {i} description: '{desc}'")
        formatted_characters = "\n".join(character_prompts)

        prompt_parts.append(character_template.format(
            num_characters=len(characters_descriptions),
            character_prompts=formatted_characters
        ))

    # 处理位置
    if location:
        location_template = """
This is the location used for the background (Ignore the locations in characters'/subjects' prompt above!)
It is critical to use this exact location: '{location}'"""
        prompt_parts.append(location_template.format(location=location))

    # 处理风格
    if style_description:
        style_template = """
This is the aesthetic/visual/artistic style to use (ignore any other style or visual aesthetic mentioned above);
ONLY extract the aesthetic/visual/artistic styles from this prompt and nothing about the subjects/location/objects/etc. within it:
'{style}'

Make sure to rewrite the prompt to conform the character/subject/location/etc. to these colors and visual styles and produce a strong style transfer.
Emphasize the updated styles early in the prompt and if applicable, include them as parts of the character/subject descriptions.
It's critical to use this exact style unless the user says otherwise.
When describing the scene and colors, it's critical that they conform to this style."""
        prompt_parts.append(style_template.format(style=style_description))

    # 处理姿势
    if pose_description:
        pose_template = """
This is the pose to use (ignore any other pose mentioned above);
ONLY extract the pose from this prompt and nothing about the subjects/location/style/etc. within it:
'{pose}'

Make sure to rewrite the prompt to conform to these poses and produce a pose transfer in a way that makes sense for the user's chosen subject.
Describe the pose of the character/subject in the image in detail so someone could draw it without the original reference.
Do not detail any of the actual objects or subjects in the scene. Purely describe the pose and action.
Emphasize the updated poses early in the prompt, but do so only concisely.
Make sure to overwrite any other poses mentioned above unless the user says otherwise."""
        prompt_parts.append(pose_template.format(pose=pose_description))

    # 处理用户指令
    if user_instructions:
        instruction_template = """
User Instructions: '{instructions}'"""
        prompt_parts.append(instruction_template.format(instructions=user_instructions))

    # 添加通用的结尾部分
    ending_template = """
Describe this picture in great detail, thinking of all the details someone would need to recreate that picture without seeing it.
Make sure to not exceed more than 200 words in this prompt.
Do not include details that do not conform to the user's specified characters/subjects, location, style, or pose.

Don't generate images, just write text. Do not prompt the user for more information. Only return the new prompt."""
    prompt_parts.append(ending_template)

    # 组合所有部分
    final_prompt = "\n".join(prompt_parts)
    return final_prompt


async def _send_request(
    url: str,
    payload,
    auth_token=None,
    cookies=None
):
    """统一处理HTTP请求（异步版本）"""
    headers = DEFAULT_HEADERS.copy()
    if auth_token:
        headers['authorization'] = auth_token
    if cookies:
        headers['Cookie'] = cookies
    try:
        response = await requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        return {"error": f"HTTP error: {str(e)}"}
    except Exception as e:
        return {"error": f"Request failed: {str(e)}"}


async def generate_image_fx(
    prompt: str,
    seed=None,
    aspect_ratio="IMAGE_ASPECT_RATIO_LANDSCAPE",
    image_number: int = 2,
    auth_token=None,
):
    """生成图像API封装（返回base64数据）"""
    payload = {
        'userInput': {
            'candidatesCount': image_number,
            'prompts': [prompt],
            **({'seed': seed} if seed is not None else {})
        },
        'clientContext': {'tool': 'BACKBONE'},
        'modelInput': {'modelNameType': 'IMAGEN_3_1'},
        'aspectRatio': aspect_ratio
    }

    response = await _send_request(
        'https://aisandbox-pa.googleapis.com/v1:runImageFx',
        payload,
        auth_token=auth_token
    )

    if 'error' in response:
        return response

    output = {
        "images": [],
        "metadata": {
            "prompt": prompt,
            "seed": seed,
            "aspect_ratio": aspect_ratio
        }
    }

    if 'imagePanels' in response:
        for panel in response['imagePanels']:
            for img_data in panel['generatedImages']:
                try:
                    output['images'].append({
                        "base64": img_data['encodedImage'],
                        "seed": img_data.get('seed'),
                        "format": "png"
                    })
                except KeyError as e:
                    print(f"Invalid image data format: {e}")
    return output


async def main(args: Args) -> Output:
    "提示词生成"
    # 解析输入参数
    style_key = args.params['style']
    character_list = []
    image1 = args.params['image1']
    image2 = args.params['image2']
    character_list = [image1, image2]
    style_info = DEFAULT_STYLE_PROMPT_DICT.get(style_key, DEFAULT_STYLE_PROMPT_DICT['贺卡'])
    instruction = args.params['extra']
    prompt = generate_imagen3_character_prompt(
        characters_descriptions=character_list,
        style_description=style_info,
        user_instructions=instruction
    )
    params = {"prompt": prompt}

    # 构建标准化输出
    return params


In [ ]:
from gradio_client import Client, handle_file

client = Client("http://0.0.0.0/")
result = client.predict(
		image_input1=handle_file(r'E:\Download\[pixiv] Trans Tribune (Wataya) - 64.jpg'),
		image_input2=None,
		style_key="贺卡",
		additional_text="/img A layered papercut Valentine's Day card depicting a young woman with fair skin and long black hair styled in pigtails.  She has small, pointed ears and small red horns. Her large red eyes are accentuated with pastel pink highlights. She wears a small, white papercut bra with pink heart stickers.   The woman is sitting on a red papercut surface, her legs spread apart. The entire scene is crafted from paper, ribbons, and stickers, with a soft, romantic color palette of pastel pinks, purples, and oranges. The style is whimsical and charming, heavily embellished with glitter. The background is a sunset scene, created using layered papercuts and ribbons.  The overall aesthetic is that of a handcrafted Valentine's Day card.",
		api_name="/generate_images"
)
print(result)